In [47]:
# Ищем A, B из j - инварианта
def invariant_to_param(F, j):    
    # A = (3*j) / (1728 -j); B = (2 * j) / (1728 - j)
    j = int(j)
    A = (3 * j) * F(1/(1728 -j))
    B = (2 * j) * F(1/(1728 -j))
    return A, B

In [48]:
# Ищем параметры кривой
def get_curve_params():
    z = 0
    while True:
        z += 1
        T = 1 + 2*z
        p = 4*z**2 + 1
        r = 4*z**2 - 2*z + 1

        if not p.is_prime():
            continue
        D = T**2 - 4*p
        if D > 0:
            continue
        Q = QuadraticField(D)

        hp = Q.hilbert_class_polynomial()
        try:
            j0 = hp.roots()[0][0]
        except IndexError:
            continue

        F = GF(p)
        A,B = invariant_to_param(F, j0)
        E = EllipticCurve(F, [A, B])

        if r == E.cardinality():
            break

    k = 6
    Fk = GF(p**k)
    Ek = EllipticCurve(Fk, [A, B])
    print('---------------------------------Parametres--------------------------------------------')
    print('z =', z)
    print('D =', D)
    print('p =', p)
    print('j0 % p =', j0 % p)
    print('---------------------------------------------------------------------------------------')
    print('\n')
    return E,Ek,r,D,T,k,p

In [49]:
# Инициализация группы кручения
torsion_group = []

In [50]:
# Выбираем точку P и Pk порядка r
def GetP(E,Ek,r,D,T):
    P = E.random_point()
    while (True):
        if (P.order() == r):
            break
        P = E.random_point()
    P = Ek(P[0], P[1], P[2])
    return P

# выбираем точку PK порядка r на кривой E(F_q), q = p^k
def GetPK(E,Ek,r,D,T):
    multiplicator = Ek.cardinality()
    while gcd(multiplicator, r) != 1:
        multiplicator //= r

    random_point = Ek.random_point()
    PK = multiplicator * random_point
    return PK

In [51]:
# Ищем группу кручения
def find_torison_gr(E,Ek,r,D,T):
    P=GetP(E,Ek,r,D,T)
    print('P=',P)
    PK=GetPK(E,Ek,r,D,T)
    print('PK=',PK)
    print('Getting table torsion_group:')
    for i in range(0, r):
        torsion_group.append(i * (P + PK))
        print(i, '* (', P, '+', PK, ') =', i * (P + PK))
        print('_________________________________________________________________________________________________')
    

In [52]:
# Проверка свойств спаривания Вейля
def weil():        
    S1 = choice(torsion_group)
    S2 = choice(torsion_group)
    S = choice(torsion_group)
    T = choice(torsion_group)
    print(S1, S2, S, T)
    
    print('-------Weil\'s check-------')
    print( 'Property 1: e(S1+S2,T)=e(S1,T)*e(S2,T) ', (S1 + S2).weil_pairing((T), r) == (S1).weil_pairing((T), r) * (S2).weil_pairing((T), r) )
    print( 'Property 2: e(S,T1+T2)=e(S,T1)*e(S,T2) ', (T).weil_pairing((S1 + S2), r) == (T).weil_pairing((S1), r) * (T).weil_pairing((S2), r) )
    print( 'Property 3: e(S,S)=1 ', (S).weil_pairing(S, r) == 1 )
    print( 'Property 4: e(S,T)=e(S,T)^(-1) ', (S).weil_pairing((T), r) == (1/(T).weil_pairing((S), r)) )
    S = Ek(0)
    print( 'Property 5: e(S,T)=1 ,S=0, T-any ', (S).weil_pairing(T, r) == 1 )
    S = choice(torsion_group)
    T = Ek(0)
    print( 'Property 6: e(S,T)=1 ,S-any, T=0 ', (S).weil_pairing(T, r) == 1 )
    print('\n')


In [53]:
# Проверка свойств спаривания Тейта
def tate():
    S1 = choice(torsion_group)
    S2 = choice(torsion_group)
    S = choice(torsion_group)
    T = choice(torsion_group)
    print(S1, S2, S, T)
    
    print('-------Tate\'s check-------')
    print( 'Property 1: e(S1+S2,T)=e(S1,T)*e(S2,T) ', (S1 + S2).tate_pairing((T), r, k) == (S1).tate_pairing((T), r, k) * (S2).tate_pairing((T), r, k) )
    print( 'Property 2: e(S,T1+T2)=e(S,T1)*e(S,T2) ', (T).tate_pairing((S1 + S2), r, k) == (T).tate_pairing((S1), r, k) * (T).tate_pairing((S2), r, k) )
    S = Ek(0)
    print( 'Property 3: e(S,T)=1 ,S=0, T-any ', (S).tate_pairing(T, r, k) == 1 )
    S = choice(torsion_group)
    T = Ek(0)
    print( 'Property 4: e(S,T)=1 ,S-any, T=0 ', (S).tate_pairing(T, r, k) == 1 )
    print('\n')

In [54]:
# Установка ключа на основе алгоритма Диффи-Хеллмана
def diffi_helm(r,k):
    P = Q = None
    while P == Q:
        P = choice(torsion_group)
        Q = choice(torsion_group)
    print(P)
    print(Q)
    print(p)
    a=r
    b=r
    c=r
    while a==r or b==r or c==r:
        a = randint(1, p)
        b = randint(1, p)
        c = randint(1, p)

    print('a, b, c =', a, b, c)

    PA = a*P
    QA = a*Q
    PB = b*P
    QB = b*Q
    PC = c*P
    QC = c*Q

    # Получение ключей
    Alice_key = ((PB).tate_pairing((QC), r, k))**a
    Bob_key = ((PA).tate_pairing((QC), r, k))**b
    Charlie_key = ((PA).tate_pairing((QB), r, k))**c
    print('A_key=',Alice_key)
    print('B_key=',Bob_key)
    print('C_key=',Charlie_key)
    print('Equal\'s check:', Alice_key == Bob_key and Alice_key == Charlie_key)


In [55]:
# Вывод всех параметров
E,Ek,r,D,T,k,p=get_curve_params()
find_torison_gr(E,Ek,r,D,T)
weil()
tate()
diffi_helm(r,k)

---------------------------------Parametres--------------------------------------------
z = 2
D = -43
p = 17
j0 % p = 15
---------------------------------------------------------------------------------------


P= (0 : 16 : 1)
PK= (9*z6^5 + 10*z6^4 + z6^3 + 5*z6^2 + 7*z6 + 7 : 11*z6^5 + 14*z6^4 + 10*z6^3 + 12*z6^2 + 2*z6 + 11 : 1)
Getting table torsion_group:
0 * ( (0 : 16 : 1) + (9*z6^5 + 10*z6^4 + z6^3 + 5*z6^2 + 7*z6 + 7 : 11*z6^5 + 14*z6^4 + 10*z6^3 + 12*z6^2 + 2*z6 + 11 : 1) ) = (0 : 1 : 0)
_________________________________________________________________________________________________
1 * ( (0 : 16 : 1) + (9*z6^5 + 10*z6^4 + z6^3 + 5*z6^2 + 7*z6 + 7 : 11*z6^5 + 14*z6^4 + 10*z6^3 + 12*z6^2 + 2*z6 + 11 : 1) ) = (9*z6^4 + 7*z6^3 + 16*z6^2 + 10*z6 + 4 : 11*z6^5 + 11*z6^4 + 15*z6^3 + 11*z6^2 + z6 + 5 : 1)
_________________________________________________________________________________________________
2 * ( (0 : 16 : 1) + (9*z6^5 + 10*z6^4 + z6^3 + 5*z6^2 + 7*z6 + 7 : 11*z6^5 + 14*z6